# Download short-cadence lightcurves

In [1]:
import lightkurve as lk
import pandas as pd
import os
from tqdm import tqdm

/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/config/__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to /Users/azib/.lightkurve/cache. Please move all the files in the legacy directory /Users/azib/.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


## Target lists
- Obtained from [here](https://tess.mit.edu/public/target_lists/target_lists.html).

In [2]:
data = pd.read_csv('../targetlists/all_targets_S028_v1.csv',skiprows=5)
data.head()

,TICID,Camera,CCD,Tmag,RA,Dec
0,12621262,1,3,10.06,335.4806,-17.7932
1,12649449,1,3,10.07,335.5289,-16.8613
2,12652062,1,3,8.82,335.5836,-16.6859
3,12652442,1,3,10.28,335.7810,-17.6114
4,12652642,1,3,9.96,335.8042,-16.8779


In [ ]:
ids = data.TICID
os.makedirs('../sc-data/',exist_ok=True)

for i in tqdm(ids):
    try:
        lc = lk.search_lightcurve(f'TIC {i}',sector=28,author='SPOC').download()
        lc.to_fits(f"../sc-data/TIC{i}.fits",overwrite=True)
    except Exception as e:
        print(f"Error downloading TIC {i}")
        print(e)
        pass
        

  0%|                                                              | 3/20000 [00:56<112:39:45, 20.28s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 3 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
  0%|                                                              | 10/20000 [02:28<74:05:58, 13.34s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
  0%|▏                                                             | 45/20000 [10:58<98:45:50, 17.82s/it]/Users/azib/micromamba/envs/nets/lib/python

Error downloading TIC 38877693
'NoneType' object has no attribute 'to_fits'


  6%|███▋                                                      | 1281/20000 [6:43:26<77:27:57, 14.90s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
  7%|███▊                                                      | 1314/20000 [6:55:46<99:18:15, 19.13s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
  7%|███▊                                                      | 1322/20000 [6:57:32<88:19:00, 17.02s/it]/Users/azib/micromamba/envs/nets/lib/python

Error downloading TIC 44577667
'NoneType' object has no attribute 'to_fits'


  8%|████▌                                                     | 1559/20000 [8:14:59<25:41:48,  5.02s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
  8%|████▋                                                     | 1598/20000 [8:20:04<35:59:58,  7.04s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
  8%|████▋                                                     | 1610/20000 [8:22:02<35:57:29,  7.04s/it]/Users/azib/micromamba/envs/nets/lib/python

...

 28%|███████████████▌                                        | 5561/20000 [32:59:56<179:50:32, 44.84s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
 28%|███████████████▌                                        | 5562/20000 [32:59:58<129:22:59, 32.26s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
 28%|███████████████▊                                         | 5563/20000 [33:00:02<94:19:19, 23.52s/it]/Users/azib/micromamba/envs/nets/lib/python

Error downloading TIC 238196512
'NoneType' object has no attribute 'to_fits'


 55%|███████████████████████████████▎                         | 10971/20000 [35:52:53<3:50:29,  1.53s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
 55%|██████████████████████████████▋                         | 10973/20000 [35:53:07<11:46:21,  4.69s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
 55%|███████████████████████████████▎                         | 10986/20000 [35:53:54<7:43:59,  3.09s/it]/Users/azib/micromamba/envs/nets/lib/python

Error downloading TIC 255559489
'NoneType' object has no attribute 'to_fits'


/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
 56%|███████████████████████████████▉                         | 11220/20000 [36:03:06<4:22:09,  1.79s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
 56%|████████████████████████████████                         | 11250/20000 [36:04:08<4:24:35,  1.81s/it]/Users/azib/micromamba/envs/nets/lib/python3.8/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. On